In [ ]:
!pip install lark mathutils pyarrow
import ifcopenshell
import ifcopenshell.geom
import ifcopenshell.util.element
import re
import georeference_ifc
from util import shape_to_polygons
import geopandas as gpd
import pandas as pd
from geopandas import GeoSeries

settings = ifcopenshell.geom.settings()
settings.set(settings.USE_WORLD_COORDS, True)
# settings.set(settings.USE_BREP_DATA, True) <-- does not work any longer somehow :shrug:
settings.set(settings.USE_PYTHON_OPENCASCADE, True)

In [ ]:
def shape_to_polygons(shape):
    from OCC.Core.TopoDS import TopoDS_Shape, TopoDS_Compound, TopoDS_Builder, topods
    from OCC.Core.GeomProjLib import geomprojlib
    from OCC.Core.BRep import BRep_Tool
    from OCC.Core.BRepTools import breptools, BRepTools_WireExplorer
    from OCC.Core.TopExp import TopExp_Explorer
    from OCC.Core.TopAbs import TopAbs_FACE, TopAbs_WIRE, TopAbs_EDGE, TopAbs_VERTEX
    from OCC.Core.gp import gp_Pln, gp_Pnt, gp_Dir
    from OCC.Core.Geom import Geom_Plane
    from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeFace, BRepBuilderAPI_MakeWire, BRepBuilderAPI_MakeEdge
    from OCC.Core.ShapeFix import ShapeFix_Wire

    from shapely.geometry import Polygon

    polygons = []
    heights = []
    exp1 = TopExp_Explorer(shape.geometry, TopAbs_FACE)
    polygon_extracted = False
    max_z = -float("inf")
    min_z = float("inf")
    while exp1.More() and not polygon_extracted:
        face = exp1.Current()
        wire = breptools.OuterWire(face)
        fix = ShapeFix_Wire()
        fix.Load(wire)
        fix.Perform()
        exp2 = BRepTools_WireExplorer(wire)
        points = []
        while exp2.More():
            vertex = exp2.CurrentVertex()
            pnt = BRep_Tool.Pnt(vertex)
            points.append([pnt.X(), pnt.Y()])
            max_z = max(max_z, pnt.Z())
            min_z = min(min_z, pnt.Z())
            exp2.Next()
        polygon = Polygon(points)
        if polygon.area > 0:  # and polygon.is_valid:
            polygons.append(polygon)
            height = max_z - min_z
            if abs(height) < 0.1:
                heights.append(None)
            else:
                heights.append(height)
            max_z = -float("inf")
            min_z = float("inf")
            polygon_extracted = True
        exp1.Next()
    return polygons, heights

In [ ]:
def extract_types(ifc_file_name: str, type: str, write_georeferenced_file: bool = False):
    ifc_file = ifcopenshell.open(ifc_file_name)
    items: list = ifc_file.by_type(type)

    georeference_ifc.set_mapconversion_crs(
        ifc_file=ifc_file,
        target_crs_epsg_code="EPSG:5243",
        eastings=86183.117831,
        northings=-303063.093919,
        orthogonal_height=293.700012207031,
        x_axis_abscissa=0.325568154457152,
        x_axis_ordinate=0.945518575599318,
        scale=1.0,
    )
    if write_georeferenced_file:
        fn_output = re.sub(r"\.ifc$", f"_georeferenced.ifc", ifc_file_name)
        ifc_file.write(fn_output)
        print(f"output written to {fn_output}")
    map_conversion, projected_crs = georeference_ifc.get_mapconversion_crs(ifc_file=ifc_file)

    return items, map_conversion, projected_crs

In [ ]:
def dict_to_string(d: dict) -> dict:
    return {str(ki): dict_to_string(di) if isinstance(di, dict) else str(di) for (ki, di) in d.items()}


def convert_to_2d(items: list, map_conversion, projected_crs) -> gpd.GeoDataFrame:
    rotation = georeference_ifc.get_rotation(map_conversion)
    print(f"Rotation is: {rotation:.1f}° (degrees(atan2(map_conversion.XAxisOrdinate, map_conversion.XAxisAbscissa)) ")
    polygons = []
    names = []
    metadatas = []
    heights = []
    for item in items:
        if item.Representation is not None:
            try:
                shape = ifcopenshell.geom.create_shape(settings, inst=item)
                shape_polygons, shape_heights = shape_to_polygons(shape)
                polygons += shape_polygons
                names += [item.Name for _ in shape_polygons]
                metadata = ifcopenshell.util.element.get_psets(item)
                # parquet (sanely) only supports keys to be strings...
                metadata = dict_to_string(metadata)
                metadatas += [metadata for _ in shape_polygons]
                heights += shape_heights
            except:
                pass

    footprint = GeoSeries(polygons, crs=projected_crs.Name)
    footprint = footprint.rotate(
        rotation,
        origin=(
            0,
            0,
        ),
        use_radians=False,
    )
    footprint = footprint.translate(map_conversion.Eastings, map_conversion.Northings, 0)
    footprint = footprint.scale(map_conversion.Scale if map_conversion.Scale else 1.0)
    heights = [h * map_conversion.Scale if map_conversion.Scale and h else h for h in heights]
    return gpd.GeoDataFrame({"name": names, "metadata": metadatas, "height": heights}, geometry=footprint)

In [ ]:
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer, format_color


def display(items_list):
    threejs_renderer = JupyterRenderer(size=(500, 500))
    for items in items_list:
        for item in items:
            if item.Representation is not None:
                shape = ifcopenshell.geom.create_shape(settings, inst=item)
                r, g, b, alpha = shape.styles[0]
                color = format_color(int(abs(r) * 255), int(abs(g) * 255), int(abs(b) * 255))
                threejs_renderer.DisplayShape(
                    shape.geometry, shape_color=color, transparency=True, opacity=alpha, render_edges=True
                )

    threejs_renderer.Display()

In [ ]:
files = [(0, "02-55-5505-100_EG.ifc")]

for level, file_name in files:
    print(f"{level=}")
    spaces, spaces_map_conversion, spaces_projected_crs = extract_types(file_name, "IFCSpace")
    df_spaces = convert_to_2d(spaces, spaces_map_conversion, spaces_projected_crs)
    df_spaces["type"] = "space"

    doors, doors_map_conversion, doors_projected_crs = extract_types(file_name, "IfcDoor")
    df_doors = convert_to_2d(doors, doors_map_conversion, doors_projected_crs)
    df_doors["type"] = "door"

    opening, opeining_map_conversion, opeining_projected_crs = extract_types(file_name, "IfcOpeningElement")
    df_opeining = convert_to_2d(opening, opeining_map_conversion, opeining_projected_crs)
    df_opeining["type"] = "opening"

    df = gpd.GeoDataFrame(pd.concat([df_doors, df_spaces, df_opeining], ignore_index=True), crs=df_spaces.crs)
    df["level"] = level
    df.to_parquet(file_name.replace(".ifc", ".parquet"))

    # data :party:
    # display([spaces, doors])